In [1]:
# final project code
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 1. 常數與基本設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Poincaré best constant for (0,1)^3 with mean-zero condition: C_P = 1/pi
C_poincare = 1.0 / torch.pi

# Sobolev best constant in R^3: C_S = 4^{1/3} / pi^{11/12}
C_sobolev = (4.0 ** (1.0 / 3.0)) / (torch.pi ** (11.0 / 12.0))

# 取樣點數
N_samples = 4096

這裡我們考慮兩個不同的constrain在domain $\Omega=(0,1)^{3}$。一個是 Poincare inequality for p=2，另一個是 Sobolev inequality:
\begin{align}\tag{1}
  \Vert u\Vert_{2,\Omega}\leq\frac{1}{\pi}\Vert\nabla u\Vert_{2,\Omega},\qquad \text{with}\quad \frac{1}{\vert\Omega\vert}\int_{\Omega}u(x)\,dx,
\end{align}
and
\begin{align}\tag{2}
  \Vert u\Vert_{6,\Omega}\leq\frac{4^{\frac{1}{3}}}{\pi^{\frac{11}{12}}}\Vert\nabla u\Vert_{2,\Omega}.
\end{align}

In [3]:
# 2. 定義網路 u_theta(x)
#    - input dimension: 3 (x1, x2, x3)
#    - hidden layers: 3 層, 每層寬度 64, sigmoid activation
#    - output dimension: 1
class SigmoidNet3D(nn.Module):
    def __init__(self, in_dim=3, hidden_dim=64, num_hidden_layers=3, out_dim=1):
        super().__init__()
        layers = []
        # input layer
        layers.append(nn.Linear(in_dim, hidden_dim))
        layers.append(nn.Sigmoid())
        # hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Sigmoid())
        # output layer
        layers.append(nn.Linear(hidden_dim, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        # x: (N, 3)
        return self.net(x)  # (N, 1)

model = SigmoidNet3D().to(device)

In [4]:
# 3. 取樣 (0,1)^3 上的點，近似積分
def sample_points_on_cube(n_samples):
    """
    Uniformly sample n_samples points in Omega = (0,1)^3.
    """
    # shape: (n_samples, 3)
    x = torch.rand(n_samples, 3, device=device, requires_grad=True)
    return x

In [5]:
# 4. 計算約束相關的 norm 與 loss
def constraint_loss(model, n_samples=N_samples):
    # 取樣點
    x = sample_points_on_cube(n_samples)  # (N, 3), requires_grad=True

    # u_theta(x)
    u = model(x)  # (N, 1)
    u = u.squeeze(-1)  # 變成 (N,)

    # 1) 平均值條件: (1/|Omega|) ∫ u = 0
    # 對 (0,1)^3, |Omega| = 1, 所以 integral ≈ mean
    mean_u = u.mean()
    mean_zero_penalty = mean_u ** 2   # 想要 mean_u -> 0

    # 2) 計算 gradient ∇u (用 autograd)
    grad_u = torch.autograd.grad(
        outputs=u,
        inputs=x,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]  # shape: (N, 3)

    # L2 norm of u over Omega (approx via Monte Carlo)
    # \|u\|_{2,\Omega} ≈ (∫ |u|^2)^{1/2} ≈ (mean(|u|^2))^{1/2}
    L2_u = torch.sqrt((u ** 2).mean() + 1e-12)

    # L6 norm of u over Omega
    # \|u\|_{6,\Omega} ≈ (mean(|u|^6))^{1/6}
    L6_u = ((u.abs() ** 6).mean() + 1e-12) ** (1.0 / 6.0)

    # L2 norm of gradient
    # \|\nabla u\|_{2,\Omega} ≈ (mean(|∇u|^2))^{1/2}
    grad_sq = (grad_u ** 2).sum(dim=1)  # (N,), each is |∇u(x)|^2
    L2_grad = torch.sqrt(grad_sq.mean() + 1e-12)

    # 3) Poincaré constraint: ||u||_2 <= (1/pi) ||∇u||_2
    # 用 ReLU 懲罰違反的部分: max(0, ||u||_2 - C_poincare * ||∇u||_2)
    poincare_violation = torch.relu(L2_u - C_poincare * L2_grad)
    poincare_penalty = poincare_violation ** 2

    # 4) Sobolev constraint: ||u||_6 <= C_sobolev * ||∇u||_2
    sobolev_violation = torch.relu(L6_u - C_sobolev * L2_grad)
    sobolev_penalty = sobolev_violation ** 2

    # constraint loss
    total_constraint_loss = mean_zero_penalty + poincare_penalty + sobolev_penalty

    # total_loss = task_loss + alpha * total_constraint_loss
    return total_constraint_loss, {
        "mean_u": mean_u.item(),
        "L2_u": L2_u.item(),
        "L6_u": L6_u.item(),
        "L2_grad": L2_grad.item(),
        "poincare_violation": poincare_violation.item(),
        "sobolev_violation": sobolev_violation.item(),
    }


**Note** 理論上的不等式應該是要完全滿足，不過我們採用soft constraint。另外，為了讓違反越來越小，我們在loss裡用了它們的平方。

In [6]:
# 5. 簡單的訓練 loop
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for step in range(1000):
    optimizer.zero_grad()
    loss, info = constraint_loss(model, N_samples)
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"step {step:4d}  loss = {loss.item():.4e}  "
              f"mean_u = {info['mean_u']:.2e}, "
              f"poincare_violation = {info['poincare_violation']:.2e}, "
              f"sobolev_violation = {info['sobolev_violation']:.2e}")

step    0  loss = 4.5381e-01  mean_u = 3.89e-01, poincare_violation = 3.89e-01, sobolev_violation = 3.89e-01
step  100  loss = 8.4966e-05  mean_u = -1.36e-03, poincare_violation = 9.07e-04, sobolev_violation = 9.07e-03
step  200  loss = 7.7598e-05  mean_u = -1.25e-05, poincare_violation = 0.00e+00, sobolev_violation = 8.81e-03
step  300  loss = 7.1569e-05  mean_u = -1.64e-05, poincare_violation = 0.00e+00, sobolev_violation = 8.46e-03
step  400  loss = 6.4438e-05  mean_u = 1.10e-05, poincare_violation = 0.00e+00, sobolev_violation = 8.03e-03
step  500  loss = 5.6389e-05  mean_u = 2.37e-05, poincare_violation = 0.00e+00, sobolev_violation = 7.51e-03
step  600  loss = 4.7623e-05  mean_u = -7.15e-06, poincare_violation = 0.00e+00, sobolev_violation = 6.90e-03
step  700  loss = 3.8405e-05  mean_u = -7.71e-05, poincare_violation = 0.00e+00, sobolev_violation = 6.20e-03
step  800  loss = 2.9172e-05  mean_u = 3.62e-05, poincare_violation = 0.00e+00, sobolev_violation = 5.40e-03
step  900  los